In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import glob

In [3]:
# 컬럼명 정의
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current', 'label']

# 정상 데이터
normal_dir = '../realtime/normal/'
normal_files = glob.glob(os.path.join(normal_dir, '*.csv'))
normal_dfs = [pd.read_csv(file, names=column_names, header=None) for file in normal_files]
normal_data = pd.concat(normal_dfs, ignore_index=True)

# 아크 데이터
arc_dir = '../realtime/arc/'
arc_files = glob.glob(os.path.join(arc_dir, '*.csv'))
arc_dfs = [pd.read_csv(file, names=column_names, header=None) for file in arc_files]
arc_data = pd.concat(arc_dfs, ignore_index=True)

# 전체 병합 및 셔플
train_df = pd.concat([normal_data, arc_data], ignore_index=True)
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [4]:
# 파생 피처
def add_features(df):
    df['voltage_diff'] = df['voltage'].diff().fillna(0).abs()
    df['current_diff'] = df['current'].diff().fillna(0).abs()
    df['voltage_ma'] = df['voltage'].rolling(5).mean().bfill()
    df['current_ma'] = df['current'].rolling(5).mean().bfill()
    df['power'] = df['voltage'] * df['current']
    df['power_diff'] = df['power'].diff().fillna(0).abs()
    return df

df = add_features(train_df).dropna()

In [5]:
features = ['voltage', 'current', 'voltage_diff', 'current_diff',
            'voltage_ma', 'current_ma', 'power', 'power_diff']

X_raw = df[features].values
y_raw = df['label'].values.astype(int)

# 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)

In [6]:
# 시퀀스 생성
def create_sequences(data, labels, seq_len=8):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(labels[i+seq_len - 1])
    return np.array(X), np.array(y)

seq_len = 8
X_seq, y_seq = create_sequences(X_scaled, y_raw, seq_len)
X_seq = X_seq.reshape((X_seq.shape[0], seq_len, len(features)))

In [7]:
# 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, stratify=y_seq, random_state=42)

In [9]:
inputs = Input(shape=(seq_len, len(features)))
x = Conv1D(64, 3, activation='relu', padding='same')(inputs)
x = MaxPooling1D(2)(x)
x = Dropout(0.3)(x)
x = Conv1D(128, 3, activation='relu', padding='same')(x)
x = MaxPooling1D(2)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 학습
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=64, callbacks=[early_stop])

Epoch 1/50
512/512 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.7551 - loss: 0.5776 - val_accuracy: 0.7690 - val_loss: 0.5221
Epoch 2/50
512/512 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.7719 - loss: 0.5342 - val_accuracy: 0.7839 - val_loss: 0.5028
Epoch 3/50
512/512 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.7875 - loss: 0.5036 - val_accuracy: 0.7891 - val_loss: 0.4960
Epoch 4/50
512/512 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.7847 - loss: 0.5007 - val_accuracy: 0.7899 - val_loss: 0.4905
Epoch 5/50
512/512 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.7877 - loss: 0.4990 - val_accuracy: 0.7924 - val_loss: 0.4849
Epoch 6/50
512/512 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.7909 - loss: 0.4920 - val_accuracy: 0.7933 - val_loss: 0.4800
Epoch 7/50
512/512 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.7933 - loss: 0.4900 - val_accuracy: 0.7933 - val_loss: 0.4789
Epoch 8/50
512/512 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.7950 - loss: 0.4783 - val_accu

In [10]:
# 모델 및 스케일러 저장
model.save('./model/cnn1d_model.h5')
joblib.dump(scaler, './model/scaler_cnn.joblib')

['./model/scaler_cnn.joblib']

In [2]:
import tensorflow as tf

model = tf.keras.models.load_model('./model/cnn1d_model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 핵심 설정: 구버전 연산자만 사용하도록 제한
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]  

# (선택) float16 경량화도 가능
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

with open("./model/cnn1d_model_compatible.tflite", "wb") as f:
    f.write(tflite_model)


ModuleNotFoundError: No module named 'tensorflow'